In [ ]:
#Scrape and download all links to the local file system due to http speed limit
import urllib.request
html_doc = ''
def init():
    with urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_Greek_mythological_figures') as response:
        html_doc = response.read()

In [ ]:
from bs4 import BeautifulSoup

f = open("wiki/Greek-index.html", 'r')
html_doc =  f.read()
soup = BeautifulSoup(html_doc, 'html.parser')
all_links = soup.find_all('a')
print(len(all_links))

In [ ]:
#Download all of the relevant web pages from all the links
def download_pages():
    counter = 0
    for link in all_links:
        if(link.has_attr('href') and "/wiki/" in link['href']):
            if counter/len(all_links) % 10 == 0:
                print(str(counter/len(all_links) * 100) + '%')
            href = link['href']
            if '.jpg' or '.svg' or '.png' in href:
                continue
            urllib.request.urlretrieve("https://en.wikipedia.org"+href, href[href.rfind('/') + 1:] + '.html')
    href = "/wiki/List_of_Greek_mythological_figures"
    urllib.request.urlretrieve("https://en.wikipedia.org" + href, href[href.rfind('/') + 1:] + '.html')

In [ ]:
import unicodedata
def strip_accents(s):
   return ''.join(c for c in unicodedata.normalize('NFD', s)
                  if unicodedata.category(c) != 'Mn')

In [ ]:
soup.h3.span.text
soup.h3.next_sibling
sibling = soup.h3
counter = 0
h3_tag = soup.h3
h3_sibling_content = soup.h3.next_sibling.next_sibling

#Go back 1 sibling to be able to grab the first h3 tag into the list
clean_siblings = [s for s in h3_tag.previous_sibling.next_siblings if (str(s).strip() != "" and (s.name == "h3" or s.name == "div" or s.name == "table" or s.name=="ul"))]
clean_siblings = clean_siblings[:-7]
clean_siblings.remove(clean_siblings[5])

it = iter(clean_siblings)
heading_content_dict= {}
for sibling in it:
    heading_content_dict[sibling.text.replace("[edit]","").replace("/"," & ")] = next(it)
print(len(heading_content_dict))
print(heading_content_dict.keys())

In [ ]:
#f = open("wiki/Hera.html",'r')
#soup = BeautifulSoup(str(f.read()), 'html.parser')

def get_data_info_box(soup):
    info_box = soup.findAll("table", { "class" : "infobox" })
    tr_list = []
    for e in info_box:
        tr_list.extend(e.findAll("tr"))
    return tr_list

def scrape_info_box(info_box_data_tr, divinity_name):
    parents = {}
    children = {}
    
    for tr in info_box_data_tr:
        if tr.th and (tr.th.text == "Children" or tr.th.text == "Parents" or tr.th.text == "Offspring"):
            print(tr.th.text + ":")
            st = re.sub(r'\s(or|and)\s', ',',tr.td.text) #Get rid of 'or' 'and' stop words
            st = strip_accents(st)
            result = [x.strip().lower() for x in re.split(";|,|\n",re.sub(r'(\[|\().*?(\]|\))','',st)) if x != '']
            if tr.th.text == "Children" or tr.th.text == "Offspring" or tr.th.text == "Parents":
                    #master_dict.setdefault(vr[vr.rfind("/") + 1:].lower(), {})
                if divinity_name in master_dict:
                    master_dict[divinity_name] = dict(master_dict[divinity_name],**{str(tr.th.text).lower(): result})
                else:
                    master_dict[divinity_name] = {str(tr.th.text).lower(): result}
                        
                if tr.th.text == "Children" or tr.th.text == "Offspring":
                    children[divinity_name] = result
                elif tr.th.text == "Parents":
                    parents[divinity_name] = result
            print(result)
    return (parents, children)

In [ ]:
def scrape_page(uri_fragment, fill_in_missing = False):
    f = open(uri_fragment[1:] + ".html", 'r')
    sub_doc = f.read()
    soup_major_sub = BeautifulSoup(str(sub_doc), 'html.parser')

    info_box_tr = get_data_info_box(soup_major_sub)
    
    uri_fragment_clean = re.sub(r'(\[|\().*?(\]|\))','', uri_fragment).strip().replace('_','')
           
    #wiki/Pallas
    divinity_name = strip_accents(uri_fragment_clean[uri_fragment_clean.rfind("/") + 1:].lower()) #Pallas
    

    mappings_uri[uri_fragment] = divinity_name  
    mappings_name[divinity_name] = uri_fragment
    
    pair = scrape_info_box(info_box_tr, divinity_name)
    
    print()
    return pair
    

In [ ]:
master_dict = {}
scraped_set = set()
mappings_uri = {} #uri to lowercase name
mappings_name = {} #lowercase name to uri

In [ ]:
import re

doc = heading_content_dict["Major gods and goddesses"]
soup_major = BeautifulSoup(str(doc), 'html.parser')
values = soup_major.find_all("td")

for td in values:
    if not td.a.img:
        vr = td.a['href']
        print(vr)
        
        tvr = re.sub(r'(\[|\().*?(\]|\))','', vr).strip().replace('_','')
        scraped_set.add(tvr[tvr.rfind("/") + 1:])
        
        result = scrape_page(vr)
        major_parents = result[0]
        major_children = result[1]



In [ ]:
print(master_dict)

In [ ]:
scraped_set.clear()

In [ ]:
import re

doc = heading_content_dict["Primordial deities"]
soup_prime = BeautifulSoup(str(doc), 'html.parser')
values = soup_prime.find_all("tr")

prime_parents = {}
prime_children = {}

for tr in values:
    if tr.contents[3].a:
        
        vr = tr.contents[3].a['href']
        print(vr)
        tvr = re.sub(r'(\[|\().*?(\]|\))','', vr).strip().replace('_','')

        scraped_set.add(tvr[tvr.rfind("/") + 1:])
        
        result = scrape_page(vr)
        
        prime_parents = result[0]
        prime_children = result[1]
        
        
        
        #prime_mappings_uri[tr.contents[3].a['href']] = vr[vr.rfind("/") + 1:].lower()
        #prime_mappings_name[vr[vr.rfind("/") + 1:].lower()] = tr.contents[3].a['href']

def id_missing_genealogy(scraped_set, parents_dict, children_dict):
    id_missing_parents = []
    id_missing_children = []
    for p in scraped_set:
        if p not in parents_dict:
            #print("No Parent(s): " + p)
            id_missing_parents.append(p)
        if p not in children_dict:
            #print("No Children(s):" + p)
            id_missing_children.append(p)
    return (id_missing_parents, id_missing_children)

id_missing_result = id_missing_genealogy(scraped_set, prime_parents, prime_children )
missing_parents = id_missing_result[0]
missing_children = id_missing_result[1]

        

In [ ]:
#from py2neo import Graph
print(mappings_name)

In [ ]:
import string
def fill_in_missing(uri, index, mapping_uri):
    parents_mapping = {} #current god to its parents
    children_mapping = {} #current god to its children
    incomplete_results = {}
    
    print("Uri: " + uri)
    f = open(uri[1:] + '.html','r')
    sub_doc = f.read()

    soup_t_sub = BeautifulSoup(str(sub_doc), 'html.parser')
    b = soup_t_sub.findAll("p")
        
    for i in b:
        st = re.sub(r'\<[^>]*\>' ,'',str(i))
        
        st = re.sub(r'(\[|\().*?(\]|\))','', st)
        translator = str.maketrans('', '', ",")
    
        st = st.translate(translator)
        st = strip_accents(st)
        regex = ""

        #k = re.search(r"(\b[A-Z]+\w*\b)\s+(?:\b[a-z]*\b\s+)*(born|progeny|daughter|father|mother|son|offspring)\s?(?:.*?)\s?of\s?(?:.*?)\s?(\b[A-Z]+[a-z]+\b)(?:\s+and\s+)?(?:\b[a-z]*\b\s+)*(\b[A-Z]+[a-z]+\b)?", st)
        matches = re.finditer(r"(\b[A-Z]+\w*\b)\s+(?:\b[a-z]*\b\s+)*(\bborn|progeny|daughter|father|mother|son|offspring\b)\s?(?:.*?)\s?(?:of|by)\s?(?:.*?)\s?(\b[A-Z]+[a-z]+\b)(?:\s+and\s+)?(?:\b[a-z]*\b\s+)*(\b[A-Z]+[a-z]+\b)?", st)
        for k in matches:
            divinity_name = mapping_uri[uri].lower()
            a = str(k.group(1)).lower()
            b = str(k.group(2)).lower()
            c = str(k.group(3)).lower()
            d = str(k.group(4)).lower()
            if a.lower() == divinity_name:
                #wiki/Pallas
                #pallas daughter of _ and _
                if(b in ('daughter','born','son', 'offspring','progeny')):
                    parents_mapping.setdefault(a,[]).extend([c, d])
                #pallas mother of _ and _
                elif(k.group(2) in ('father','mother')):
                    children_mapping.setdefault(a,[]).extend([c, d])

            elif c.lower() == divinity_name or d.lower() == divinity_name:
                #x son of pallas and _
                if(k.group(2) in ('daughter','born','son', 'offspring','progeny')):
                    children_mapping.setdefault(divinity_name.title(),[]).append(k.group(1))
                #y father of pallas and _
                elif(k.group(2) in ('father','mother')):
                    parents_mapping.setdefault(divinity_name.title(),[]).append(k.group(1))
            else:
                incomplete_results.setdefault(uri,[]).append([index, k]) 
                print("Incomplete: " + uri + " : " + k.group(0))
    #print()
    return {"Parents": parents_mapping, "Children": children_mapping, "Incomplete": incomplete_results}

In [ ]:
#print(master_dict)
master_incomplete_results = []
def processIncompleteList(missingList, mapping_name, mapping_uri):
    results = {}
    incomplete_results = [] #Record those with incomplete results
    for idx, p in enumerate(missingList):
        print("Current Index: " + str(idx))
        results = fill_in_missing(mapping_name[p.lower()], idx, mapping_uri)
        master_dict.setdefault(p.lower(),{})
        if len(results["Incomplete"]) > 0:
            incomplete_results.append(results)
        if(len(results["Parents"]) > 0):
            print(results["Parents"])
            master_dict[p.lower()].setdefault("parents",[]).extend(list(results["Parents"].values())[0])
        if(len(results["Children"]) > 0):
            master_dict[p.lower()].setdefault("children",[]).extend(list(results["Children"].values())[0])
        print()
    return incomplete_results

In [ ]:
#Prime processing
master_incomplete_results += processIncompleteList(missing_parents, mappings_name, mappings_uri)

In [ ]:
print(master_dict['uranus'])

In [ ]:
#Ignore Ourea
#Tatarus should be father of Giants
print(len(master_incomplete_results))
for i in master_incomplete_results:
    print(i)
    print()

In [ ]:

#master_dict['tartarus']['children'] = ["giants"]
master_dict['achlys']['parents'].append('nyx')
#master_incomplete_results.clear() #discard rest of the results

In [ ]:
print(len(master_dict))
print(master_dict)

In [ ]:
doc = heading_content_dict["Titans and Titanesses"]
soup_titan = BeautifulSoup(str(doc), 'html.parser')
values = soup_titan.find_all("tr")

values = values[2:]

titan_parents = {}
titan_children = {}

scraped_set.clear()

for tr in values:
    if len(tr.contents) < 4:
        continue
    if tr.contents[3].a:
          
        vr = tr.contents[3].a['href']
        print(vr)
        tvr = re.sub(r'(\[|\().*?(\]|\))','', vr).strip().replace('_','')

        scraped_set.add(tvr[tvr.rfind("/") + 1:])
        
        result = scrape_page(vr)
        
        titan_parents = result[0]
        titan_children = result[1]
        
id_missing_result = id_missing_genealogy(scraped_set, titan_parents, titan_children )
missing_parents = id_missing_result[0]
missing_children = id_missing_result[1]        

In [ ]:
master_incomplete_results += processIncompleteList(missing_parents, mappings_name, mappings_uri)

In [ ]:
master_incomplete_results += processIncompleteList(missing_children, mappings_name, mappings_uri)

In [ ]:
master_dict['prometheus']

In [ ]:
doc = heading_content_dict['Gigantes and other "giants"']
                           
soup_titan = BeautifulSoup(str(doc), 'html.parser')
values = soup_titan.find_all("li")
for li in values:
    print(li.find_all("a",href=True))


In [ ]:
#heading_content_dict['Gigantes and other "giants"']
#